In [ ]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu"
from model import build_visual_model
from caffezoo.googlenet import GoogleNet
from caffezoo.vgg import VGG
from lasagne import layers, updates
import theano.tensor as T
from helpers import load_word_embedding
import numpy as np
from lasagnekit.easy import (BatchOptimizer, build_batch_iterator,
                           InputOutputMapping)
from lasagnekit.nnet.capsule import Capsule
from lasagnekit.datasets.imagenet import ImageNet
import pickle
from collections import OrderedDict

import lshash
#!https://pypi.python.org/pypi/lshash/0.0.4dev


In [ ]:
# imagenet data
batch_size = 10
size = (224, 224)
imagenet = ImageNet(size=size, nb=batch_size)

# word model
word2vec_filename = "data/glove.6B.50d.pkl"
word2vec = load_word_embedding(word2vec_filename)
word2int = {word: i for i, word in enumerate(word2vec.keys())}  # NOQA
int2word = {i: word for i, word in enumerate(word2vec.keys())}  # NOQA
int2vec = np.array([word2vec[int2word[i]]
                  for i in range(len(word2vec))])
int2vec /= np.sqrt((int2vec**2).sum(axis=1))[:, None]  # unit norm

size_embedding = len(word2vec.values()[0])

In [ ]:
imagenet.load()

In [ ]:
import pickle
model = pickle.load(open("out/model.pkl"))

In [ ]:
cls = VGG
size = (224, 224)
visual_model_base = cls(layer_names=["input"], input_size=size,
                        resize=False)
visual_model_base._load()

In [ ]:
g = VGG(layer_names=["fc6"], resize=False)
g._load()

In [ ]:
x=visual_model_base.preprocess(imagenet.X)
print(x.shape)

In [ ]:
model["embed"].W.get_value()

In [ ]:
import theano
embed = theano.function([model["input"].input_var],
                        layers.get_output(model["embed"]))

In [ ]:
e = embed(x)
print(e)
#print(e.sum())
#e /= np.sqrt(e**2).sum(axis=1)[:, None]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for i, w in enumerate(e):
    plt.imshow(imagenet.X[i])
    plt.show()
    sim = (int2vec * w).sum(axis=1)
    print(int2word[sim.argmax()])